# Loading the Data

In [33]:
import numpy as np
import pandas as pd 
from sklearn.datasets import load_breast_cancer

In [34]:
cancer = load_breast_cancer()

In [35]:
type(cancer)

sklearn.utils._bunch.Bunch

In [36]:
X = cancer.data
y = cancer.target

In [37]:
cancer.feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [38]:
cancer.target_names

array(['malignant', 'benign'], dtype='<U9')

In [39]:
print(cancer.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [40]:
X[0]

array([1.799e+01, 1.038e+01, 1.228e+02, 1.001e+03, 1.184e-01, 2.776e-01,
       3.001e-01, 1.471e-01, 2.419e-01, 7.871e-02, 1.095e+00, 9.053e-01,
       8.589e+00, 1.534e+02, 6.399e-03, 4.904e-02, 5.373e-02, 1.587e-02,
       3.003e-02, 6.193e-03, 2.538e+01, 1.733e+01, 1.846e+02, 2.019e+03,
       1.622e-01, 6.656e-01, 7.119e-01, 2.654e-01, 4.601e-01, 1.189e-01])

In [41]:
y[0]

0

In [42]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42, stratify=y)

# Pipeline without Dimensionality Reduction

In [43]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [44]:
pipe = Pipeline([("scaler", StandardScaler()),
                 ("SVM", SVC(random_state=42))])

In [45]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('SVM', SVC(random_state=42))])

In [46]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipe,X_train,y_train,cv=5, scoring="accuracy")
acc = scores.mean()
print(acc)

0.9714285714285715


In [52]:
X.shape

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

## Dimensionality Reduction - PCA
Principal component analysis (dimensonality reduction)

In [48]:
from sklearn.decomposition import PCA
# n_components is the number of components to keep
pca_step = ("pca", PCA(n_components=5))
pca_pipe = Pipeline([("scaler", StandardScaler()),
                     pca_step,
                     ("SVM", SVC(random_state=42))])

In [49]:
pca_pipe.fit(X_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=5)),
                ('SVM', SVC(random_state=42))])

In [50]:
scores = cross_val_score(pca_pipe,X_train,y_train,cv=5, scoring="accuracy")
acc = scores.mean()
print(acc)

0.9714285714285715


In [59]:
pca_obj = pca_pipe.named_steps["pca"]
ex = pca_obj.explained_variance_ratio_
ex

array([0.44413492, 0.18944618, 0.09543356, 0.06724689, 0.0551769 ])

In [60]:
ex.sum()

0.8514384620342296

# Hyper-parameter Tuning

In [61]:
from sklearn.model_selection import GridSearchCV

params = [
          {"pca__n_components":[5,4,6] , 
           "SVM__C": [0.1,1,2,3],
           "SVM__kernel":["linear","rbf"]},
          {"SVM__gamma":[0.1,1,2,3],
           "SVM__kernel": ["poly","rbf",'sigmoid']}
]

In [62]:
grid = GridSearchCV(pca_pipe, param_grid=params, cv = 3)

In [63]:
grid.fit(X_train,y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('pca', PCA(n_components=5)),
                                       ('SVM', SVC(random_state=42))]),
             param_grid=[{'SVM__C': [0.1, 1, 2, 3],
                          'SVM__kernel': ['linear', 'rbf'],
                          'pca__n_components': [5, 4, 6]},
                         {'SVM__gamma': [0.1, 1, 2, 3],
                          'SVM__kernel': ['poly', 'rbf', 'sigmoid']}])

In [64]:
print(f"Best Score : {grid.best_score_}")
print(f"Best Parameters: {grid.best_params_}")

Best Score : 0.9736551644010688
Best Parameters: {'SVM__C': 3, 'SVM__kernel': 'rbf', 'pca__n_components': 5}


In [65]:
final_model = grid.best_estimator_

In [66]:
from sklearn.metrics import accuracy_score
y_pred = final_model.predict(X_test)
acc = accuracy_score(y_test,y_pred)
acc

0.9473684210526315

# Summary

In [ ]:
"""
1. What is Machine learning
2. Numpy
3. Pandas
4. Matplotlib
5. Scikit
6. Linear Regression
7. Classification
8. Clustering
9. PCA - Hyperpamater Tuning
"""